In [2]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.6 MB/s eta 0:00:00


In [3]:
!pip install flask pyngrok pycryptodome

In [4]:
from flask import Flask, request, render_template_string
import hashlib
from pyngrok import conf, ngrok
import os

app = Flask(__name__)
os.environ["NGROK_AUTH_TOKEN"] = "2ujCdDkaOVBkBmrqIRJACyVCwRD_2nF1MMtFrfkdkjTpKxkJT"
conf.get_default().auth_token = os.environ["NGROK_AUTH_TOKEN"]


# Function to hash data using SHA-256
def hash_sha256(data: bytes) -> str:
    """
    Hashes the given bytes data using SHA-256 algorithm.

    Args:
        data: The bytes data to be hashed.

    Returns:
        A hexadecimal string representation of the SHA-256 hash.
    """
    sha256_hash = hashlib.sha256()
    sha256_hash.update(data)
    return sha256_hash.hexdigest()

# Function to hash data using SHA-512
def hash_sha512(data: bytes) -> str:
    """
    Hashes the given bytes data using SHA-512 algorithm.

    Args:
        data: The bytes data to be hashed.

    Returns:
        A hexadecimal string representation of the SHA-512 hash.
    """
    sha512_hash = hashlib.sha512()
    sha512_hash.update(data)
    return sha512_hash.hexdigest()

# HTML web interface
HTML = '''
<!DOCTYPE html>
<html lang="vi">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Băm dữ liệu bằng SHA-256 và SHA-512</title>
    <style>
        body {
            font-family: 'Segoe UI', sans-serif;
            background: linear-gradient(to right, #232526, #ce1313);
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            margin: 0;
            padding: 20px;
            box-sizing: border-box;
        }
        .container {
            background: #ffffffcc;
            backdrop-filter: blur(5px);
            padding: 30px 40px;
            border-radius: 16px;
            box-shadow: 0 8px 24px rgba(0, 0, 0, 0.2);
            width: 100%;
            max-width: 600px; /* Increased max-width to accommodate two hash results */
            box-sizing: border-box;
        }
        h2 {
            text-align: center;
            margin-bottom: 25px;
            color: #333;
        }
        p {
            margin: 10px 0;
            font-weight: bold;
        }
        input[type="file"], select, button {
            width: 100%;
            padding: 10px 12px;
            margin-top: 6px;
            border-radius: 8px;
            border: 1px solid #ccc;
            box-sizing: border-box;
            font-size: 14px;
        }
        button {
            background: linear-gradient(to right, #232526, #ce1313);
            color: white;
            font-weight: bold;
            border: none;
            cursor: pointer;
            margin-top: 20px;
            transition: background 0.3s ease;
        }
        button:hover {
            background: linear-gradient(to right, #5b86e5, #36d1dc);
        }
        .result-box {
            background-color: #f0f0f0;
            border: 1px solid #ddd;
            border-radius: 8px;
            padding: 15px;
            margin-top: 25px;
            word-break: break-all;
            font-family: 'Consolas', 'Courier New', monospace;
            color: #333;
            font-size: 0.9em;
        }
        .result-box strong {
            display: block;
            margin-bottom: 5px;
            color: #555;
        }
        .error {
            color: red;
            font-weight: bold;
            margin-top: 15px;
            text-align: center;
        }
    </style>
</head>
<body>
    <div class="container">
        <h2>Băm dữ liệu bằng SHA-256/SHA-512</h2>
        <form method="POST" enctype="multipart/form-data">
            <p>Chọn file:
            <input type="file" name="file" required></p>

            <p>Thuật toán băm:
            <select name="algorithm" required>
                <option value="sha256">SHA-256</option>
                <option value="sha512">SHA-512</option>
            </select></p>

            <button type="submit">Thực hiện băm</button>
        </form>

        {% if original_hashed_result %}
            <div class="result-box">
                <strong>Kết quả băm gốc ({{ algorithm }}):</strong>
                <p>{{ original_hashed_result }}</p>
            </div>
            <div class="result-box">
                <strong>Kết quả băm đã sửa đổi ({{ algorithm }} - thêm một ký tự):</strong>
                <p>{{ modified_hashed_result }}</p>
            </div>
        {% endif %}

        {% if error %}
            <div class="error">{{ error }}</div>
        {% endif %}
    </div>
</body>
</html>
'''

@app.route('/', methods=['GET', 'POST'])
def home():
    """
    Handles the home route, displaying the form and processing file uploads for hashing.
    It calculates hashes for both the original and a slightly modified version of the data.
    """
    error = None
    original_hashed_result = None
    modified_hashed_result = None
    algorithm_used = None

    if request.method == 'POST':
        if 'file' not in request.files:
            error = "Không có file nào được chọn."
        else:
            f = request.files['file']
            if f.filename == '':
                error = "Không có file nào được chọn."
            else:
                algorithm = request.form.get('algorithm', 'sha256') # Default to sha256
                original_data = f.read() # Read the original file content as bytes

                # Create a slightly modified version of the data
                # For demonstration, we'll append a single newline byte.
                # In a real scenario, this could be any small, intentional change.
                modified_data = original_data + b'\n'

                try:
                    if algorithm == 'sha256':
                        original_hashed_result = hash_sha256(original_data)
                        modified_hashed_result = hash_sha256(modified_data)
                        algorithm_used = "SHA-256"
                    elif algorithm == 'sha512':
                        original_hashed_result = hash_sha512(original_data)
                        modified_hashed_result = hash_sha512(modified_data)
                        algorithm_used = "SHA-512"
                    else:
                        error = "Thuật toán băm không hợp lệ."
                except Exception as e:
                    error = f"Lỗi trong quá trình băm: {e}"

    # Render the HTML template, passing the results and error messages
    return render_template_string(
        HTML,
        original_hashed_result=original_hashed_result,
        modified_hashed_result=modified_hashed_result,
        algorithm=algorithm_used,
        error=error
    )

try:
    ngrok.kill()
    public_url = ngrok.connect(5000).public_url
    print(f"📌 Truy cập web tại: {public_url}")
except Exception as e:
    print(f"Lỗi khi khởi tạo ngrok: {e}")
    public_url = "Ngrok không thể khởi tạo."

app.run(port=5000, use_reloader=False)


📌 Truy cập web tại: https://471f-34-125-79-167.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [31/May/2025 02:13:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 02:13:41] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 02:14:06] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 02:15:01] "POST / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [31/May/2025 02:15:03] "GET / HTTP/1.1" 200 -


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
